Classes

In [56]:
from __future__ import annotations
import math
import random
from typing import List, Dict, Tuple, Callable
from dataclasses import dataclass, field
from enum import Enum
from copy import deepcopy

# Enum para definir o tipo de otimização (MIN ou MAX)
class TipoOtimizacao(Enum):
    MIN = 0
    MAX = 1

# Classe que representa uma partícula no algoritmo PSO
@dataclass
class Particula:
    tamanho: int  # Tamanho da partícula
    solucao: List[float] = field(default_factory=list)  # Vetor com a solução do problema
    deslocamento: List[float] = field(default_factory=list)  # Vetor de deslocamento
    valor_fitness: float = 0  # Valor do fitness da partícula
    melhor_global: Particula = None  # Melhor partícula na vizinhança
    melhor_pessoal: Particula = None  # Melhor localização da partícula ao longo do tempo
    dominio: List[Tuple[int, int]] = field(default_factory=list)  # Domínio (min, max) de cada variável
    tipo_otimizacao: TipoOtimizacao = TipoOtimizacao.MIN  # Tipo de otimização (MIN ou MAX)

    def __post_init__(self):
        if self.solucao: return
        if self.dominio:
            # Inicializa a posição e velocidade das partículas
            self.solucao = [random.uniform(self.dominio[i][0], self.dominio[i][1]) for i in range(self.tamanho)]
            self.deslocamento = [random.uniform(self.dominio[i][0], self.dominio[i][1]) for i in range(self.tamanho)]
        else:
            # Inicializa a posição e velocidade das partículas aleatoriamente
            self.solucao = [random.random() for _ in range(self.tamanho)]
            self.deslocamento = [random.random() for _ in range(self.tamanho)]
        self.melhor_pessoal = self
        # Define o fitness inicial com infinito
        self.valor_fitness = math.inf if self.tipo_otimizacao == TipoOtimizacao.MIN else -math.inf

    def verifica_melhor(self, outra: Particula):
        if self.tipo_otimizacao == TipoOtimizacao.MIN:
            return self.valor_fitness < outra.valor_fitness
        return self.valor_fitness > outra.valor_fitness

Algoritmo principal

In [57]:
def pso(peso_inercia: float, coef_cognitivo: float, coef_social: float, tam_populacao: int, max_iteracoes: int, funcao_fitness: Callable[[Particula], float], dominio: List[Tuple[int, int]] = None):

    populacao = [Particula(tamanho=2, dominio=dominio) for _ in range(tam_populacao)]
    melhor_global = deepcopy(populacao[0])
    # Atualiza o melhor pessoal de cada partícula
    for iteracao in range(max_iteracoes):
        for particula in populacao:
            particula.valor_fitness = funcao_fitness(particula)
            if particula.verifica_melhor(particula.melhor_pessoal):
                particula.melhor_pessoal = deepcopy(particula)

        # Atualiza o melhor global
        for particula in populacao:
            if particula.verifica_melhor(melhor_global):
                melhor_global = particula
        melhor_global = deepcopy(melhor_global)

        # Define o melhor global para todas as partículas
        for particula in populacao:
            particula.melhor_global = melhor_global

        # Atualiza a posição da partícula
        for particula in populacao:
            for i in range(particula.tamanho):
                particula.deslocamento[i] = peso_inercia * particula.deslocamento[i] + coef_cognitivo * random.random() * (particula.melhor_pessoal.solucao[i] - particula.solucao[i]) + coef_social * random.random() * (particula.melhor_global.solucao[i] - particula.solucao[i])
                particula.solucao[i] += particula.deslocamento[i]

    # Imprime os resultados em uma tabela
    print("Iteração | Solução | Fitness")
    for iteracao in range(max_iteracoes):
        print(f"{iteracao+1:6} | {[f'{i:.4f}' for i in melhor_global.solucao]} | {melhor_global.valor_fitness:.3f}")

    return melhor_global

Fitness

In [58]:
def funcao_fitness(p: Particula):
    x1 = p.solucao[0]
    x2 = p.solucao[1]
    return -((x1 ** 2) / 2) - ((x1 ** 2) / 2) + (x1 - 2 * x2) + math.cos(x1 * x2)

In [59]:
peso_inercia = 0.4
coef_cognitivo = 2
coef_social = 2
tam_populacao = 20000
max_iteracoes = 200
dominio = [
    (-1000, 1000),  # x1
    (-1000, 1000),  # x2
]

solucao = pso(peso_inercia, coef_cognitivo, coef_social, tam_populacao, max_iteracoes, funcao_fitness, dominio)

Iteração | Solução | Fitness
     1 | ['-477496990891696918234524949111737526636706740924579840.0000', '837.6923'] | -228003376310625298041018117720215365847410973402180184692591719443230196357989828109583550915413872750886912.000
     2 | ['-477496990891696918234524949111737526636706740924579840.0000', '837.6923'] | -228003376310625298041018117720215365847410973402180184692591719443230196357989828109583550915413872750886912.000
     3 | ['-477496990891696918234524949111737526636706740924579840.0000', '837.6923'] | -228003376310625298041018117720215365847410973402180184692591719443230196357989828109583550915413872750886912.000
     4 | ['-477496990891696918234524949111737526636706740924579840.0000', '837.6923'] | -228003376310625298041018117720215365847410973402180184692591719443230196357989828109583550915413872750886912.000
     5 | ['-477496990891696918234524949111737526636706740924579840.0000', '837.6923'] | -22800337631062529804101811772021536584741097340218018469259171944323019635